In [118]:
import pandas as pd
from collections import Counter
import re
import pickle

In [99]:
DATA_PATH = ''

In [167]:
extra_data = pd.read_csv(DATA_PATH+'extra_data.csv')
punctuations = extra_data['punctuations'].dropna().to_list()
stop_words = extra_data['stop_words'].dropna().to_list()
suffix_u = extra_data['suffix_u'].dropna().to_list()
suffix = extra_data['suffix'].dropna().to_list()
ar = extra_data['ar'].dropna().to_list()
pr = extra_data['pr'].dropna().to_list()
verb_post = extra_data['verb_post'].dropna().to_list()
verb_pre = extra_data['verb_pre'].dropna().to_list()
verbs = extra_data['verbs'].dropna().to_list()
df = pd.read_csv(DATA_PATH+'IR_data.csv')
mokasar = pd.read_csv(DATA_PATH+'mokasar.csv')
df

,id,content,url
0,1,به گزارش ایسنا، پس از استعفا علی نکویی رئیس شا...,https://www.isna.ir/news/99010100077/حواشی-در-...
1,2,به گزارش ایسنا، امیر محسنی با بیان اینکه این ل...,https://www.isna.ir/news/98122922468/ثبت-نام-ب...
2,3,آرش فرهادیان در گفت و گو با ایسنا، درباره آخری...,https://www.isna.ir/news/99010200541/فرهادیان-...
3,4,به گزارش ایسنا، فدراسیون بین المللی شنا قانونی...,https://www.isna.ir/news/99010200528/فناوری-وی...
4,5,به گزارش ایسنا، فدراسیون جهانی صعودهای ورزشی (...,https://www.isna.ir/news/99010200510/تعویق-زما...
...,...,...,...
6995,6996,**به گزارش ایسنا،** سیگار کشیدن، علت عمده‌ برو...,https://www.isna.ir/news/98092015395/اصلی-ترین...
6996,6997,**به گزارش ایسنا،** ​​​انسولین هورمونی است که ...,https://www.isna.ir/news/98092015327/انواع-دیا...
6997,6998,فاطمه احمدی - موزه‌دار - در گفت‌وگو با ایسنا، ...,https://www.isna.ir/news/98092216157/پنجمین-مو...
6998,6999,**به گزارش ایسنا،** سیگار کشیدن، علت عمده‌ برو...,https://www.isna.ir/news/98092015395/اصلی-ترین...


In [101]:
mokasar = mokasar.set_index('jam').T.to_dict('dict')

In [102]:
suffix

['ها', 'تر', 'ترین', 'گان', 'ان', 'ات', 'ی', 'ای', 'یی']

In [103]:
df = df.dropna()

In [104]:
def tokenize(text):
  tokens = set([word for word in text.split()])
  return tokens

In [105]:
def delete_suffix(word, suffix,u=1):
  word_end = word.index(suffix[0])
  if u and '\u200c' in word and word.index('\u200c') == word_end - 1 and word.endswith(suffix):
    word = word[:word_end-1]
  elif not u and suffix in word and word.replace(suffix,'') in all_words and word.endswith(suffix):
    if suffix == 'گان':
      word = word.replace(suffix,'ه')
    else:
      word = word.replace(suffix,'')
  return word

In [106]:
def delete_punctuations(word):
  for punc in punctuations:
    if punc in word:
      word = word.replace(punc," ")
  return word

In [107]:
def delete_none_words(word):
    persian_pattern =  "^[\u0643\u0623\u0629\u0624\u0621\u0626\u0622\u0627\u0628\u067E\u062A-\u062C\u0686\u062D-\u0632\u0698\u0633-\u063A\u0641\u0642\u06A9\u06AF\u0644-\u0648\u06CC]+$"
    if re.match(persian_pattern,word):
        return word
    return ""

In [108]:
def normalize_letters(word):
    for i in len(ar):
        if ar[i] in word:
            word.replace(ar[i],pr[i])

In [109]:
def normalize_mokasar(word):
    if word in mokasar.keys():
        word = mokasar[word]['mofrad']
    return word

In [110]:
def normalize_verb(verb):
    for v in verbs:
        if verb in v:
            for p in verb_pre:
                if verb.startswith(p):
                    verb = verb.replace(p,'')
            for p in verb_post:
                if verb.endswith(p):
                    verb = verb.replace(p,'')
    return verb

In [111]:
text_list = df.content.to_list()
all_words = tokenize(" ".join(text_list))
# for i in range(len(all_words)):
#     all_words[i] = delete_none_words(all_words[i])
all_words = [delete_none_words(a_w) for a_w in all_words if delete_none_words(a_w)]

In [112]:
len(all_words)

27686

In [113]:
words_freq = dict(Counter(all_words))

In [114]:
len(words_freq)

27686

In [115]:
from tqdm import tqdm

In [116]:
def normalize_words(text_list):
  words_dict = {}
  for i in tqdm(range(len(text_list))):
    temp_words = text_list[i].split()
    temp_words = [w.strip() for w in temp_words if w.strip() not in stop_words]
    
    # print
    # for word in text_list[i].split():
    #   if word in stop_words:
    #     word = ""
    x = temp_words
    for j in range(len(temp_words)):
      temp_words[j] = delete_punctuations(temp_words[j])
      temp_words[j] = delete_none_words(temp_words[j])
      temp_words[j] = normalize_verb(temp_words[j])
      for s in suffix_u:
        if s in temp_words[j]:
          temp_words[j] = delete_suffix(temp_words[j],s)
          break
      for s in suffix:
        if s in temp_words[j]:
          temp_words[j] = delete_suffix(temp_words[j],s,u=0)
          break
    temp_words = [w for w in temp_words if w.strip() not in stop_words]

    words = list(map(str.strip, temp_words))
    for j in range(len(words)):
      if words[j] not in words_dict.keys():
        words_dict[words[j]] = {}
      if i not in words_dict[words[j]].keys():
        words_dict[words[j]][i] = 1
      else:
        words_dict[words[j]][i] += 1
  return words_dict

In [117]:
indexed_words = normalize_words(text_list)

100%|██████████| 6996/6996 [09:52<00:00, 11.80it/s]


In [119]:
del indexed_words['']

In [120]:
len(indexed_words)

23531

In [121]:
with open('indexed_words.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('indexed_words.pickle', 'rb') as handle:
    indexed_words = pickle.load(handle)

In [143]:
query = 'واکسیناسیون بیماری کرونا'

In [144]:
query = normalize_words([query])

100%|██████████| 1/1 [00:00<00:00, 128.15it/s]


In [145]:
query

{'واکسیناسیون': {0: 1}, 'بیماری': {0: 1}, 'کرونا': {0: 1}}

In [146]:
match = []
for q in query.keys():
    for iw in indexed_words.keys():
        if q == iw:
            print(iw)
            match.append(list(indexed_words[iw].keys()))
            print(len(match))
#     return match

واکسیناسیون
1
بیماری
2
کرونا
3


In [147]:
len(match)

3

In [148]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [149]:
mm = match[0]
for m in match:
    mm = intersection(mm,m)

In [150]:
mm

[5589,
 5681,
 5960,
 5999,
 6001,
 6011,
 6018,
 6034,
 6041,
 6123,
 6226,
 6269,
 6397,
 6404,
 6424,
 6431,
 6440,
 6502,
 6504,
 6505,
 6509]

In [151]:
df[df.id.isin(mm)]

,id,content,url
5588,5589,**به گزارش ایسنا،** دکتر کیانوش جهانپور در این...,https://www.isna.ir/news/99022417719/شناسایی-م...
5680,5681,دکتر مهرنوش رنجبر در تعریفی از گرمازدگی گفت: گ...,https://www.isna.ir/news/99032515909/یک-وضعیت-...
5959,5960,دکتر عبدالرسول همتی پنجشنبه 13 شهریور در جمع خ...,https://www.isna.ir/news/99061309635/تازه-ترین...
5998,5999,عبدالکریم گراوند پیش از ظهر امروز در نشست ستاد...,https://www.isna.ir/news/99062317412/آنها-که-ک...
6000,6001,به گزارش ایسنا و بنابر اعلام روابط عمومی کرونا...,https://www.isna.ir/news/99062317044/آثار-کرون...
6010,6011,به گزارش ایسنا، در این دیدار، دکتر مسعود حبیبی...,https://www.isna.ir/news/99062619870/رصد-روزان...
6017,6018,سعید کشمیری با اعلام این خبر گفت: در ۲۴ ساعت گ...,https://www.isna.ir/news/99062821323/۳۰۶-بیمار...
6033,6034,**به گزارش ایسنا،** دکتر سیما سادات لاری دربار...,https://www.isna.ir/news/99070201609/ظرفیت-بیم...
6040,6041,**به گزارش ایسنا،** دکتر سیما سادات لاری دربار...,https://www.isna.ir/news/99070201609/ظرفیت-بیم...
6122,6123,به گزارش ایسنا و بنابر اعلام پژوهشگاه ابن سینا...,https://www.isna.ir/news/99072820379/نقش-تغذیه...


In [168]:
dfs = {}
for word in indexed_words.keys():
    dfs[word] = len(indexed_words[word])

In [169]:
query = 'واکسیناسیون بیماری کرونا در جهان'

In [170]:
query = normalize_words([query])

100%|██████████| 1/1 [00:00<00:00, 209.52it/s]


In [171]:
query

{'واکسیناسیون': {0: 1}, 'بیماری': {0: 1}, 'کرونا': {0: 1}, 'جهان': {0: 1}}

In [172]:
match = []
for q in query.keys():
    for iw in indexed_words.keys():
        if q == iw:
            print(iw)
            match.append(list(indexed_words[iw].keys()))
            print(len(match))
#     return match

واکسیناسیون
1
بیماری
2
کرونا
3
جهان
4


In [173]:
len(match)

4

In [174]:
import math

In [175]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [176]:
mm = match[0]
for m in match:
    mm = intersection(mm,m)

In [177]:
def compute_distance(query, words):
    N = len(words)
    length = [0]*N
    scores = [0]*N
    for term in tqdm(query.keys()):
        print(term)
        print(dfs[term])
        idf = math.log(N/dfs[term])
        w_tq = (1+math.log(query[term][0])) * idf
        for docid in words[term].keys():
            w_td = (math.log(words[term][docid]) + 1)
            scores[docid]+= w_td * w_tq
            length[docid]+= (w_td**2)
    for docid in range(N):
        if scores[docid]:
            scores[docid] /= (length[docid]**0.5)
    return scores

In [178]:
scores = compute_distance(query,indexed_words)

100%|██████████| 4/4 [00:00<00:00, 847.42it/s]

واکسیناسیون
55
بیماری
1137
کرونا
1359
جهان
661


In [179]:
import numpy as np

In [180]:
# sorted_scores = np.argsort(scores)

In [195]:
def get_top_k(k, scores):
    sorted_scores = np.argsort(scores)
    a = sorted_scores[-1*k:]
    top_k = a[::-1]
    return top_k

In [196]:
top_10 = get_top_k(10,scores)

In [197]:
top_10

array([6474, 6727, 6724, 6721, 6226, 5960, 6404, 5681, 6509, 3207])

In [198]:
# for d in top_k:
print(df.loc[top_10[0]])

id                                                      6475
content    ابراهیم رضایی در ارائه گزارش به خبرنگاران به م...
url        https://www.isna.ir/news/99121209285/استان-بوش...
Name: 6474, dtype: object


# Champion List

In [199]:
def make_champions(r, indexed_words):
    champion_list = {}
    for term in indexed_words:
        doc_dict = indexed_words[term].copy()
        champions = dict(sorted(doc_dict.items(), key=lambda x: x[1], reverse=True)[:r])
        champion_list[term] = champions
    return champion_list

In [200]:
champion_list = make_champions(5,indexed_words)
champion_sims = compute_distance(query,champion_list)

100%|██████████| 4/4 [00:00<00:00, 2358.67it/s]

واکسیناسیون
55
بیماری
1137
کرونا
1359
جهان
661


In [201]:
top_10 = get_top_k(10,champion_sims)

In [202]:
top_10

array([6407, 5960, 6454, 6269, 6440, 6123, 1317,  977, 1623,   32])

In [203]:
print(df.loc[top_10[0]])

id                                                      6408
content    سیدعلی آقازاده در گفت‌وگو با ایسنا، اظهار کرد:...
url        https://www.isna.ir/news/99112014708/واکسن-کرو...
Name: 6407, dtype: object


# فاز سه